# Legacy Code to Download Yahoo Finance Data (Incomplete)

In [ ]:
import requests
import urllib.parse
import pandas as pd
from datetime import datetime

class YahooFinanceData:
    """
    A class to retrieve, hold, and analyze Yahoo Finance data for a given ticker symbol.
    
    This can be extended to incorporate financial statements, statistics, more
    advanced analysis, etc.
    """
    
    BASE_URL = "https://query1.finance.yahoo.com/v8/finance/chart/"
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) '
                      'AppleWebKit/605.1.15 (KHTML, like Gecko) '
                      'Version/17.1 Safari/605.1.15',
        'Accept-Language': 'en-GB,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept': '*/*',
    }
    
    def __init__(self, ticker, 
                 start_date=None, 
                 end_date=None, 
                 interval="1d"):
        """
        Parameters
        ----------
        ticker: str
            Ticker symbol (e.g., 'EQNR.OL').
        start_date: str or datetime, optional
            Start date for the data. If None, defaults to some suitable range.
        end_date: str or datetime, optional
            End date for the data. If None, defaults to the current date.
        interval: str, optional
            Data interval (e.g., '1d', '1wk', '1mo').
        """
        self.ticker = ticker
        self.interval = interval
        self.start_date = self._parse_date(start_date)
        self.end_date = self._parse_date(end_date, default_to_today=True)
        
        # Internal containers for fetched data
        self.fetch_history()
        
    def _parse_date(self, date_val, default_to_today=False):
        """
        Parses a date string or datetime into a UNIX timestamp integer.
        If `default_to_today` is True and date_val is None, uses today's date.
        """
        if date_val is None:
            if default_to_today:
                return int(datetime.now().timestamp())
            # Default to some older start if needed, e.g. 1 year ago:
            return int(datetime(datetime.now().year - 1, 
                                datetime.now().month, 
                                datetime.now().day).timestamp())
        
        if isinstance(date_val, str):
            date_val = datetime.fromisoformat(date_val)
        
        return int(date_val.timestamp())
    
    def fetch_history(self):
        """
        Retrieves the raw JSON data from Yahoo Finance and stores it internally.
        """
        symquote = urllib.parse.quote(self.ticker)
        url = (
            f"{self.BASE_URL}{symquote}"
            f"?formatted=true"
            f"&includeAdjustedClose=true"
            f"&interval={self.interval}"
            f"&period1={self.start_date}"
            f"&period2={self.end_date}"
            f"&events=capitalGain%7Cdiv%7Csplit"
            f"&symbol={symquote}"
            f"&userYfid=true"
            f"&lang=en-US"
            f"&region=US"
        )
        # Optionally, you can add a 'referer' key to HEADERS if needed:
        referer = f"https://finance.yahoo.com/quote/{urllib.parse.quote(self.ticker)}/history"
        local_headers = dict(self.HEADERS)
        local_headers['Referer'] = referer
        
        try:
            response = requests.get(url, headers=local_headers)
            response.raise_for_status()  # Will raise an HTTPError if the request was not successful
            self.raw_history = response.json()
        except Exception as e:
            print(f"Error while fetching history of '{self.ticker}'")
            
    def fetch_financials(self):
        # TODO
        return
    
    def fetch_statistics(self):
        # TODO
        return
    
    def fetch_analysis(self):
        # TODO
        return 
    
    def extract_basic_info(self):
        """
        Extracts basic instrument information from the internal JSON.
        Returns a dict with keys like symbol, shortName, longName, etc.
        """
        if self.raw_history is None:
            raise ValueError("No data has been fetched yet. Call fetch_data() first.")
        
        if self._basic_info is None:  # Cache the result so we don't re-parse every time
            meta = self.raw_history["chart"]["result"][0]["meta"]
            self._basic_info = {
                "symbol": meta.get("symbol"),
                "shortName": meta.get("shortName"),
                "longName": meta.get("longName"),
                "exchangeName": meta.get("exchangeName"),
                "instrumentType": meta.get("instrumentType"),
            }
        return self._basic_info
    
    def extract_price_data(self):
        """
        Extracts pricing data (timestamp, open, high, low, close, volume) 
        and returns it as a pandas DataFrame.
        """
        if self.raw_history is None:
            raise ValueError("No data has been fetched yet. Call fetch_data() first.")
        
        timestamps = self.raw_history['chart']['result'][0]['timestamp']
        quote_data = self.raw_history['chart']['result'][0]['indicators']['quote'][0]
        
        df = pd.DataFrame({
            'timestamp': timestamps,
            'open': quote_data['open'],
            'high': quote_data['high'],
            'low': quote_data['low'],
            'close': quote_data['close'],
            'volume': quote_data['volume']
        })
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
        self._price_data = df
        
        return self._price_data
    
    # Below are placeholders for future expansion:
    
    def fetch_financial_statements(self):
        """
        Placeholder method to fetch financial statements.
        For example, you can retrieve Income Statements, Balance Sheets, etc.
        from alternative Yahoo endpoints or other APIs.
        """
        pass
    
    def fetch_statistics(self):
        """
        Placeholder method to fetch additional statistics about the company,
        e.g., P/E ratio, market cap, dividends, etc.
        """
        pass
    
    def run_analysis(self):
        """
        Placeholder method to perform analysis on the retrieved data.
        You could implement moving averages, RSI calculations,
        or other technical/fundamental analytics here.
        """
        pass


In [114]:
yf_data = YahooFinanceData(
    ticker="NVDA", 
    start_date="2020-11-01", 
    end_date="2025-03-20",
    interval="1d"
)

In [115]:
yf_data.raw_history

{'chart': {'result': [{'meta': {'currency': 'USD',
     'symbol': 'NVDA',
     'exchangeName': 'NMS',
     'fullExchangeName': 'NasdaqGS',
     'instrumentType': 'EQUITY',
     'firstTradeDate': 917015400,
     'regularMarketTime': 1742414402,
     'hasPrePostMarketData': True,
     'gmtoffset': -14400,
     'timezone': 'EDT',
     'exchangeTimezoneName': 'America/New_York',
     'regularMarketPrice': 117.52,
     'fiftyTwoWeekHigh': 153.13,
     'fiftyTwoWeekLow': 75.606,
     'regularMarketDayHigh': 120.445,
     'regularMarketDayLow': 115.68,
     'regularMarketVolume': 270676243,
     'longName': 'NVIDIA Corporation',
     'shortName': 'NVIDIA Corporation',
     'chartPreviousClose': 12.534,
     'priceHint': 2,
     'currentTradingPeriod': {'pre': {'timezone': 'EDT',
       'start': 1742457600,
       'end': 1742477400,
       'gmtoffset': -14400},
      'regular': {'timezone': 'EDT',
       'start': 1742477400,
       'end': 1742500800,
       'gmtoffset': -14400},
      'post': 

In [118]:
yf_data.extract_price_data()

,timestamp,open,high,low,close,volume
0,2020-11-02 14:30:00,12.657750,12.836750,12.395000,12.580750,299652000
1,2020-11-03 14:30:00,12.708500,13.105000,12.602750,13.019500,272900000
2,2020-11-04 14:30:00,13.389250,13.827500,13.337000,13.794250,410228000
3,2020-11-05 14:30:00,14.162500,14.339000,13.991000,14.160000,319028000
4,2020-11-06 14:30:00,14.109750,14.588750,13.895000,14.562000,383348000
...,...,...,...,...,...,...
1094,2025-03-13 13:30:00,117.029999,117.760002,113.790001,115.580002,299033100
1095,2025-03-14 13:30:00,118.610001,121.879997,118.150002,121.669998,277593500
1096,2025-03-17 13:30:00,122.739998,122.889999,118.029999,119.529999,255501500
1097,2025-03-18 13:30:00,118.000000,119.019997,114.540001,115.430000,299686900


In [ ]:
ticker = "EQNR.OL"
symquote = urllib.parse.quote(ticker)

url = f"https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/EQNR.OL?merge=false&padTimeSeries=true&period1=493590046&period2=1742464799&type=quarterlyTaxEffectOfUnusualItems%2CtrailingTaxEffectOfUnusualItems%2CquarterlyTaxRateForCalcs%2CtrailingTaxRateForCalcs%2CquarterlyNormalizedEBITDA%2CtrailingNormalizedEBITDA%2CquarterlyNormalizedDilutedEPS%2CtrailingNormalizedDilutedEPS%2CquarterlyNormalizedBasicEPS%2CtrailingNormalizedBasicEPS%2CquarterlyTotalUnusualItems%2CtrailingTotalUnusualItems%2CquarterlyTotalUnusualItemsExcludingGoodwill%2CtrailingTotalUnusualItemsExcludingGoodwill%2CquarterlyNetIncomeFromContinuingOperationNetMinorityInterest%2CtrailingNetIncomeFromContinuingOperationNetMinorityInterest%2CquarterlyReconciledDepreciation%2CtrailingReconciledDepreciation%2CquarterlyReconciledCostOfRevenue%2CtrailingReconciledCostOfRevenue%2CquarterlyEBITDA%2CtrailingEBITDA%2CquarterlyEBIT%2CtrailingEBIT%2CquarterlyNetInterestIncome%2CtrailingNetInterestIncome%2CquarterlyInterestExpense%2CtrailingInterestExpense%2CquarterlyInterestIncome%2CtrailingInterestIncome%2CquarterlyContinuingAndDiscontinuedDilutedEPS%2CtrailingContinuingAndDiscontinuedDilutedEPS%2CquarterlyContinuingAndDiscontinuedBasicEPS%2CtrailingContinuingAndDiscontinuedBasicEPS%2CquarterlyNormalizedIncome%2CtrailingNormalizedIncome%2CquarterlyNetIncomeFromContinuingAndDiscontinuedOperation%2CtrailingNetIncomeFromContinuingAndDiscontinuedOperation%2CquarterlyTotalExpenses%2CtrailingTotalExpenses%2CquarterlyRentExpenseSupplemental%2CtrailingRentExpenseSupplemental%2CquarterlyReportedNormalizedDilutedEPS%2CtrailingReportedNormalizedDilutedEPS%2CquarterlyReportedNormalizedBasicEPS%2CtrailingReportedNormalizedBasicEPS%2CquarterlyTotalOperatingIncomeAsReported%2CtrailingTotalOperatingIncomeAsReported%2CquarterlyDividendPerShare%2CtrailingDividendPerShare%2CquarterlyDilutedAverageShares%2CtrailingDilutedAverageShares%2CquarterlyBasicAverageShares%2CtrailingBasicAverageShares%2CquarterlyDilutedEPS%2CtrailingDilutedEPS%2CquarterlyDilutedEPSOtherGainsLosses%2CtrailingDilutedEPSOtherGainsLosses%2CquarterlyTaxLossCarryforwardDilutedEPS%2CtrailingTaxLossCarryforwardDilutedEPS%2CquarterlyDilutedAccountingChange%2CtrailingDilutedAccountingChange%2CquarterlyDilutedExtraordinary%2CtrailingDilutedExtraordinary%2CquarterlyDilutedDiscontinuousOperations%2CtrailingDilutedDiscontinuousOperations%2CquarterlyDilutedContinuousOperations%2CtrailingDilutedContinuousOperations%2CquarterlyBasicEPS%2CtrailingBasicEPS%2CquarterlyBasicEPSOtherGainsLosses%2CtrailingBasicEPSOtherGainsLosses%2CquarterlyTaxLossCarryforwardBasicEPS%2CtrailingTaxLossCarryforwardBasicEPS%2CquarterlyBasicAccountingChange%2CtrailingBasicAccountingChange%2CquarterlyBasicExtraordinary%2CtrailingBasicExtraordinary%2CquarterlyBasicDiscontinuousOperations%2CtrailingBasicDiscontinuousOperations%2CquarterlyBasicContinuousOperations%2CtrailingBasicContinuousOperations%2CquarterlyDilutedNIAvailtoComStockholders%2CtrailingDilutedNIAvailtoComStockholders%2CquarterlyAverageDilutionEarnings%2CtrailingAverageDilutionEarnings%2CquarterlyNetIncomeCommonStockholders%2CtrailingNetIncomeCommonStockholders%2CquarterlyOtherunderPreferredStockDividend%2CtrailingOtherunderPreferredStockDividend%2CquarterlyPreferredStockDividends%2CtrailingPreferredStockDividends%2CquarterlyNetIncome%2CtrailingNetIncome%2CquarterlyMinorityInterests%2CtrailingMinorityInterests%2CquarterlyNetIncomeIncludingNoncontrollingInterests%2CtrailingNetIncomeIncludingNoncontrollingInterests%2CquarterlyNetIncomeFromTaxLossCarryforward%2CtrailingNetIncomeFromTaxLossCarryforward%2CquarterlyNetIncomeExtraordinary%2CtrailingNetIncomeExtraordinary%2CquarterlyNetIncomeDiscontinuousOperations%2CtrailingNetIncomeDiscontinuousOperations%2CquarterlyNetIncomeContinuousOperations%2CtrailingNetIncomeContinuousOperations%2CquarterlyEarningsFromEquityInterestNetOfTax%2CtrailingEarningsFromEquityInterestNetOfTax%2CquarterlyTaxProvision%2CtrailingTaxProvision%2CquarterlyPretaxIncome%2CtrailingPretaxIncome%2CquarterlyOtherIncomeExpense%2CtrailingOtherIncomeExpense%2CquarterlyOtherNonOperatingIncomeExpenses%2CtrailingOtherNonOperatingIncomeExpenses%2CquarterlySpecialIncomeCharges%2CtrailingSpecialIncomeCharges%2CquarterlyGainOnSaleOfPPE%2CtrailingGainOnSaleOfPPE%2CquarterlyGainOnSaleOfBusiness%2CtrailingGainOnSaleOfBusiness%2CquarterlyOtherSpecialCharges%2CtrailingOtherSpecialCharges%2CquarterlyWriteOff%2CtrailingWriteOff%2CquarterlyImpairmentOfCapitalAssets%2CtrailingImpairmentOfCapitalAssets%2CquarterlyRestructuringAndMergernAcquisition%2CtrailingRestructuringAndMergernAcquisition%2CquarterlySecuritiesAmortization%2CtrailingSecuritiesAmortization%2CquarterlyEarningsFromEquityInterest%2CtrailingEarningsFromEquityInterest%2CquarterlyGainOnSaleOfSecurity%2CtrailingGainOnSaleOfSecurity%2CquarterlyNetNonOperatingInterestIncomeExpense%2CtrailingNetNonOperatingInterestIncomeExpense%2CquarterlyTotalOtherFinanceCost%2CtrailingTotalOtherFinanceCost%2CquarterlyInterestExpenseNonOperating%2CtrailingInterestExpenseNonOperating%2CquarterlyInterestIncomeNonOperating%2CtrailingInterestIncomeNonOperating%2CquarterlyOperatingIncome%2CtrailingOperatingIncome%2CquarterlyOperatingExpense%2CtrailingOperatingExpense%2CquarterlyOtherOperatingExpenses%2CtrailingOtherOperatingExpenses%2CquarterlyOtherTaxes%2CtrailingOtherTaxes%2CquarterlyProvisionForDoubtfulAccounts%2CtrailingProvisionForDoubtfulAccounts%2CquarterlyDepreciationAmortizationDepletionIncomeStatement%2CtrailingDepreciationAmortizationDepletionIncomeStatement%2CquarterlyDepletionIncomeStatement%2CtrailingDepletionIncomeStatement%2CquarterlyDepreciationAndAmortizationInIncomeStatement%2CtrailingDepreciationAndAmortizationInIncomeStatement%2CquarterlyAmortization%2CtrailingAmortization%2CquarterlyAmortizationOfIntangiblesIncomeStatement%2CtrailingAmortizationOfIntangiblesIncomeStatement%2CquarterlyDepreciationIncomeStatement%2CtrailingDepreciationIncomeStatement%2CquarterlyResearchAndDevelopment%2CtrailingResearchAndDevelopment%2CquarterlySellingGeneralAndAdministration%2CtrailingSellingGeneralAndAdministration%2CquarterlySellingAndMarketingExpense%2CtrailingSellingAndMarketingExpense%2CquarterlyGeneralAndAdministrativeExpense%2CtrailingGeneralAndAdministrativeExpense%2CquarterlyOtherGandA%2CtrailingOtherGandA%2CquarterlyInsuranceAndClaims%2CtrailingInsuranceAndClaims%2CquarterlyRentAndLandingFees%2CtrailingRentAndLandingFees%2CquarterlySalariesAndWages%2CtrailingSalariesAndWages%2CquarterlyGrossProfit%2CtrailingGrossProfit%2CquarterlyCostOfRevenue%2CtrailingCostOfRevenue%2CquarterlyTotalRevenue%2CtrailingTotalRevenue%2CquarterlyExciseTaxes%2CtrailingExciseTaxes%2CquarterlyOperatingRevenue%2CtrailingOperatingRevenue&lang=en-US&region=US"

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.1 Safari/605.1.15',
    'Accept-Language': 'en-GB,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': '*/*',
    'Cookie': '_cb=BX29pkB54V0RDBXLcD; _cb_svref=https%3A%2F%2Ffinance.yahoo.com%2Fquote%2FNOKUSD%3DX%2Fhistory%2F; _chartbeat2=.1742459915365.1742464302191.1.lybfTCZZpd0xmDQUCHqKGtdZ1wy.2; _chartbeat4=t=B2ZABk-0KfbCOeEQcD-Csr3lfp8R&E=7&x=0&c=0.29&y=2067&w=670; PRF=t%3DEQNR.OL%252BNOKUSD%253DX%252BEQNR%26theme%3Dauto; _SUPERFLY_lockout=1; _chartbeat5=449|300|%2Fquote%2FEQNR.OL%2F|https%3A%2F%2Ffinance.yahoo.com%2Fquote%2FEQNR.OL%2Ffinancials%2F|Dpji8EBMwcfOC6WsWICrF1fdaoXE-||r|Dz3dvNDUwzBWCWkMEWBOqOasDV8q0s|finance.yahoo.com|; EuConsent=CQNOsMAQNOsMAAOACBNBBdFoAP_gAEPgACiQKptB9G7WTXFneXp2YPskOYUX0VBJ4MAwBgCBAcABzBIUIBwGVmAzJEyIICACGAIAIGBBIABtGAhAQEAAYIAFAABIAEgAIBAAIGAAACAAAABACAAAAAAAAAAQgEAXMBQgmAYEBFoIQUhAggAgAQAAAAAEAIgBCAQAEAAAQAAICAAIACgAAgAAAAAAAAAEAFAIEQAAIAECAotkdQAAAAAAAAAAAAQACAABAAAAAIKpgAkGpUQBFgSEhAIGEECAEQUBABQIAgAACBAAAATBAUIAwAVGAiAEAIAAAAAAAAACABAAABAAhAAEAAQIAAAAAIAAgAIBAAACAAAAAAAAAAAAAAAAAAAAAAAAAGIBAggAAABBAAQUAAAAAgAAAAAAAAAIgACAAAAAAAAAAAAAAIgAAAAAAAAAAAAAAAAAAIAAAAIAAAAgBEFgAAAAAAAAAAAAAACAABAAAAAIAAA; cmp=t=1742459891&j=1&u=1---&v=72; A1=d=AQABBMHBuWcCEK4AZJXdGpyXfRCrFc2Kd44FEgABCAEIu2ftZ_mOb2UB9qMAAAcIv8G5Z0Sd0PA&S=AQAAAqIQLh93V3EYi3PwA31vKVQ; A1S=d=AQABBMHBuWcCEK4AZJXdGpyXfRCrFc2Kd44FEgABCAEIu2ftZ_mOb2UB9qMAAAcIv8G5Z0Sd0PA&S=AQAAAqIQLh93V3EYi3PwA31vKVQ; A3=d=AQABBMHBuWcCEK4AZJXdGpyXfRCrFc2Kd44FEgABCAEIu2ftZ_mOb2UB9qMAAAcIv8G5Z0Sd0PA&S=AQAAAqIQLh93V3EYi3PwA31vKVQ; GUC=AQABCAFnuwhn7UIfpQTS&s=AQAAAMX_5ARI&g=Z7nByw'
}

financials = requests.get(url, headers=HEADERS)


financials.json()